#User Features System

**Date:** January 5, 2026  
**Goal:** Implement User Profiles, Search History, and Favorites  

---

## 📋 Features to Build:
1. ✅ User Profile System (registration, preferences)
2. ✅ Search History Tracking
3. ✅ Favorites Management
4. ✅ Integration with v2.3 Agent

---

## PART 1: Setup & Mount Drive

In [18]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
# Change to project directory
import os
os.chdir('/content/drive/MyDrive/ai_fashion_assistant_v2')

# Create v2.4 directory structure
!mkdir -p v2.4-complete/data/users
!mkdir -p v2.4-complete/notebooks
!mkdir -p v2.4-complete/src
!mkdir -p v2.4-complete/evaluation/results

print('✅ Directory structure created')
!ls -la v2.4-complete/

✅ Directory structure created
total 16
drwx------ 3 root root 4096 Jan  4 20:39 data
drwx------ 3 root root 4096 Jan  4 20:39 evaluation
drwx------ 2 root root 4096 Jan  4 20:39 notebooks
drwx------ 2 root root 4096 Jan  4 20:39 src


In [20]:
# Install dependencies (if needed)
# !pip install groq --quiet

print('✅ Dependencies ready')

✅ Dependencies ready


In [21]:
# Imports
import json
import os
from datetime import datetime
from typing import Dict, List, Optional
from dataclasses import dataclass, asdict
from pathlib import Path
from collections import defaultdict, Counter

print('✅ Imports complete')

✅ Imports complete


---

## PART 2: User Profile System

In [22]:
# Configuration
USER_DATA_DIR = Path('v2.4-complete/data/users')
USER_DATA_DIR.mkdir(parents=True, exist_ok=True)

print(f'✅ User data directory: {USER_DATA_DIR}')

✅ User data directory: v2.4-complete/data/users


In [23]:
@dataclass
class UserPreferences:
    """User preference settings"""
    style: List[str] = None  # ['casual', 'formal', 'sporty']
    size: str = 'M'  # XS, S, M, L, XL
    budget_min: int = 0
    budget_max: int = 1000
    colors: List[str] = None  # ['blue', 'black', 'white']
    categories: List[str] = None  # ['dress', 'shoes', 'accessories']

    def __post_init__(self):
        if self.style is None:
            self.style = []
        if self.colors is None:
            self.colors = []
        if self.categories is None:
            self.categories = []

@dataclass
class UserProfile:
    """Complete user profile"""
    user_id: str
    name: str
    email: Optional[str] = None
    preferences: UserPreferences = None
    created_at: str = None
    last_active: str = None

    def __post_init__(self):
        if self.preferences is None:
            self.preferences = UserPreferences()
        if self.created_at is None:
            self.created_at = datetime.now().isoformat()
        if self.last_active is None:
            self.last_active = datetime.now().isoformat()

    def to_dict(self):
        """Convert to dictionary for JSON storage"""
        data = asdict(self)
        data['preferences'] = asdict(self.preferences)
        return data

    @classmethod
    def from_dict(cls, data):
        """Create from dictionary"""
        prefs_data = data.pop('preferences')
        prefs = UserPreferences(**prefs_data)
        return cls(preferences=prefs, **data)

# Test
test_user = UserProfile(
    user_id='U001',
    name='Test User',
    email='test@example.com'
)

print('✅ User profile classes defined')
print('\nExample user:')
print(json.dumps(test_user.to_dict(), indent=2))

✅ User profile classes defined

Example user:
{
  "user_id": "U001",
  "name": "Test User",
  "email": "test@example.com",
  "preferences": {
    "style": [],
    "size": "M",
    "budget_min": 0,
    "budget_max": 1000,
    "colors": [],
    "categories": []
  },
  "created_at": "2026-01-04T20:43:30.437312",
  "last_active": "2026-01-04T20:43:30.437340"
}


In [24]:
class UserProfileManager:
    """Manages user profiles with JSON storage"""

    def __init__(self, storage_dir=USER_DATA_DIR):
        self.storage_dir = Path(storage_dir)
        self.storage_dir.mkdir(parents=True, exist_ok=True)
        self.users_file = self.storage_dir / 'users.json'
        self.users = self._load_all_users()

    def _load_all_users(self) -> Dict[str, UserProfile]:
        """Load all users from storage"""
        if self.users_file.exists():
            with open(self.users_file, 'r') as f:
                data = json.load(f)
                return {
                    uid: UserProfile.from_dict(udata)
                    for uid, udata in data.items()
                }
        return {}

    def _save_all_users(self):
        """Save all users to storage"""
        data = {uid: user.to_dict() for uid, user in self.users.items()}
        with open(self.users_file, 'w') as f:
            json.dump(data, f, indent=2)

    def create_user(self, user_id: str, name: str, email: str = None) -> UserProfile:
        """Create new user profile"""
        if user_id in self.users:
            raise ValueError(f"User {user_id} already exists")

        user = UserProfile(user_id=user_id, name=name, email=email)
        self.users[user_id] = user
        self._save_all_users()

        print(f"✅ Created user: {user_id}")
        return user

    def get_user(self, user_id: str) -> Optional[UserProfile]:
        """Get user profile"""
        return self.users.get(user_id)

    def update_user(self, user_id: str, **kwargs) -> UserProfile:
        """Update user profile fields"""
        if user_id not in self.users:
            raise ValueError(f"User {user_id} not found")

        user = self.users[user_id]
        for key, value in kwargs.items():
            if hasattr(user, key):
                setattr(user, key, value)

        user.last_active = datetime.now().isoformat()
        self._save_all_users()

        print(f"✅ Updated user: {user_id}")
        return user

    def update_preferences(self, user_id: str, **pref_kwargs) -> UserProfile:
        """Update user preferences"""
        if user_id not in self.users:
            raise ValueError(f"User {user_id} not found")

        user = self.users[user_id]
        for key, value in pref_kwargs.items():
            if hasattr(user.preferences, key):
                setattr(user.preferences, key, value)

        user.last_active = datetime.now().isoformat()
        self._save_all_users()

        print(f"✅ Updated preferences for: {user_id}")
        return user

    def delete_user(self, user_id: str) -> bool:
        """Delete user profile"""
        if user_id in self.users:
            del self.users[user_id]
            self._save_all_users()
            print(f"✅ Deleted user: {user_id}")
            return True
        return False

    def list_users(self) -> List[str]:
        """List all user IDs"""
        return list(self.users.keys())

    def get_stats(self) -> Dict:
        """Get user statistics"""
        return {
            'total_users': len(self.users),
            'users_with_preferences': sum(
                1 for u in self.users.values()
                if u.preferences.style or u.preferences.colors
            )
        }

print('✅ UserProfileManager class defined')

✅ UserProfileManager class defined


In [25]:
# Test UserProfileManager
manager = UserProfileManager()

# Create test users
print('Creating test users...')
try:
    user1 = manager.create_user('U001', 'Alice', 'alice@example.com')
    user2 = manager.create_user('U002', 'Bob', 'bob@example.com')
    user3 = manager.create_user('U003', 'Carol', 'carol@example.com')
except ValueError as e:
    print(f'⚠️  {e} (users may already exist)')

# Update preferences
print('\nUpdating preferences...')
manager.update_preferences(
    'U001',
    style=['casual', 'sporty'],
    size='S',
    budget_min=100,
    budget_max=500,
    colors=['blue', 'white']
)

manager.update_preferences(
    'U002',
    style=['formal', 'elegant'],
    size='L',
    budget_min=200,
    budget_max=800,
    colors=['black', 'gray']
)

print('\n📊 User Statistics:')
print(json.dumps(manager.get_stats(), indent=2))

print('\n👤 User 1 Profile:')
print(json.dumps(manager.get_user('U001').to_dict(), indent=2))

Creating test users...
⚠️  User U001 already exists (users may already exist)

Updating preferences...
✅ Updated preferences for: U001
✅ Updated preferences for: U002

📊 User Statistics:
{
  "total_users": 3,
  "users_with_preferences": 3
}

👤 User 1 Profile:
{
  "user_id": "U001",
  "name": "Alice",
  "email": "alice@example.com",
  "preferences": {
    "style": [
      "casual",
      "sporty"
    ],
    "size": "S",
    "budget_min": 100,
    "budget_max": 500,
    "colors": [
      "blue",
      "white"
    ],
    "categories": []
  },
  "created_at": "2026-01-04T20:39:52.036993",
  "last_active": "2026-01-04T20:43:30.480228"
}


---

## PART 3: Search History System

In [26]:
@dataclass
class SearchEntry:
    """Single search history entry"""
    query: str
    timestamp: str
    results_count: int
    top_result_id: Optional[str] = None
    response_time: float = 0.0

    def to_dict(self):
        return asdict(self)

    @classmethod
    def from_dict(cls, data):
        return cls(**data)

print('✅ SearchEntry class defined')

✅ SearchEntry class defined


In [27]:
class SearchHistory:
    """Manages user search history"""

    def __init__(self, user_id: str, storage_dir=USER_DATA_DIR):
        self.user_id = user_id
        self.storage_dir = Path(storage_dir)
        self.history_file = self.storage_dir / f'history_{user_id}.json'
        self.history = self._load_history()

    def _load_history(self) -> List[SearchEntry]:
        """Load search history from file"""
        if self.history_file.exists():
            with open(self.history_file, 'r') as f:
                data = json.load(f)
                return [SearchEntry.from_dict(entry) for entry in data]
        return []

    def _save_history(self):
        """Save search history to file"""
        data = [entry.to_dict() for entry in self.history]
        with open(self.history_file, 'w') as f:
            json.dump(data, f, indent=2)

    def add_search(
        self,
        query: str,
        results_count: int,
        top_result_id: str = None,
        response_time: float = 0.0
    ) -> SearchEntry:
        """Add new search to history"""
        entry = SearchEntry(
            query=query,
            timestamp=datetime.now().isoformat(),
            results_count=results_count,
            top_result_id=top_result_id,
            response_time=response_time
        )

        self.history.append(entry)
        self._save_history()

        return entry

    def get_recent(self, n: int = 10) -> List[SearchEntry]:
        """Get n most recent searches"""
        return self.history[-n:]

    def search_in_history(self, keyword: str) -> List[SearchEntry]:
        """Find searches containing keyword"""
        keyword_lower = keyword.lower()
        return [
            entry for entry in self.history
            if keyword_lower in entry.query.lower()
        ]

    def clear_history(self):
        """Clear all history"""
        self.history = []
        self._save_history()
        print(f"✅ Cleared history for user: {self.user_id}")

    def get_stats(self) -> Dict:
        """Get history statistics"""
        if not self.history:
            return {'total_searches': 0}

        return {
            'total_searches': len(self.history),
            'unique_queries': len(set(e.query for e in self.history)),
            'avg_results': sum(e.results_count for e in self.history) / len(self.history),
            'avg_response_time': sum(e.response_time for e in self.history) / len(self.history)
        }

print('✅ SearchHistory class defined')

✅ SearchHistory class defined


In [28]:
# Test SearchHistory
history = SearchHistory('U001')

# Add sample searches
print('Adding sample searches...')
history.add_search("blue dress", 15, "P001", 0.234)
history.add_search("running shoes", 23, "P045", 0.189)
history.add_search("summer jacket", 18, "P123", 0.201)
history.add_search("black jeans", 12, "P567", 0.178)
history.add_search("white t-shirt", 20, "P234", 0.156)

print('\n📜 Recent Searches:')
for entry in history.get_recent(5):
    print(f"  - {entry.query} ({entry.results_count} results, {entry.response_time:.3f}s)")

print('\n🔍 Search for "dress":')
dress_searches = history.search_in_history("dress")
for entry in dress_searches:
    print(f"  - {entry.query}")

print('\n📊 History Statistics:')
print(json.dumps(history.get_stats(), indent=2))

Adding sample searches...

📜 Recent Searches:
  - blue dress (15 results, 0.234s)
  - running shoes (23 results, 0.189s)
  - summer jacket (18 results, 0.201s)
  - black jeans (12 results, 0.178s)
  - white t-shirt (20 results, 0.156s)

🔍 Search for "dress":
  - blue dress
  - blue dress

📊 History Statistics:
{
  "total_searches": 10,
  "unique_queries": 5,
  "avg_results": 17.6,
  "avg_response_time": 0.1916
}


---

## PART 4: Favorites System

In [29]:
@dataclass
class FavoriteProduct:
    """Favorite product entry"""
    product_id: str
    product_name: str
    product_category: str
    added_at: str
    notes: Optional[str] = None

    def to_dict(self):
        return asdict(self)

    @classmethod
    def from_dict(cls, data):
        return cls(**data)

print('✅ FavoriteProduct class defined')

✅ FavoriteProduct class defined


In [30]:
class FavoritesManager:
    """Manages user favorites"""

    def __init__(self, user_id: str, storage_dir=USER_DATA_DIR):
        self.user_id = user_id
        self.storage_dir = Path(storage_dir)
        self.favorites_file = self.storage_dir / f'favorites_{user_id}.json'
        self.favorites: Dict[str, FavoriteProduct] = self._load_favorites()

    def _load_favorites(self) -> Dict[str, FavoriteProduct]:
        """Load favorites from file"""
        if self.favorites_file.exists():
            with open(self.favorites_file, 'r') as f:
                data = json.load(f)
                return {
                    pid: FavoriteProduct.from_dict(pdata)
                    for pid, pdata in data.items()
                }
        return {}

    def _save_favorites(self):
        """Save favorites to file"""
        data = {pid: prod.to_dict() for pid, prod in self.favorites.items()}
        with open(self.favorites_file, 'w') as f:
            json.dump(data, f, indent=2)

    def add_favorite(
        self,
        product_id: str,
        product_name: str,
        product_category: str,
        notes: str = None
    ) -> bool:
        """Add product to favorites"""
        if product_id in self.favorites:
            print(f"⚠️  Product {product_id} already in favorites")
            return False

        favorite = FavoriteProduct(
            product_id=product_id,
            product_name=product_name,
            product_category=product_category,
            added_at=datetime.now().isoformat(),
            notes=notes
        )

        self.favorites[product_id] = favorite
        self._save_favorites()

        print(f"✅ Added {product_name} to favorites")
        return True

    def remove_favorite(self, product_id: str) -> bool:
        """Remove product from favorites"""
        if product_id in self.favorites:
            name = self.favorites[product_id].product_name
            del self.favorites[product_id]
            self._save_favorites()
            print(f"✅ Removed {name} from favorites")
            return True

        print(f"⚠️  Product {product_id} not in favorites")
        return False

    def is_favorite(self, product_id: str) -> bool:
        """Check if product is favorited"""
        return product_id in self.favorites

    def get_all(self) -> List[FavoriteProduct]:
        """Get all favorites"""
        return list(self.favorites.values())

    def get_by_category(self, category: str) -> List[FavoriteProduct]:
        """Get favorites by category"""
        return [
            fav for fav in self.favorites.values()
            if fav.product_category.lower() == category.lower()
        ]

    def get_recent(self, n: int = 10) -> List[FavoriteProduct]:
        """Get n most recently added favorites"""
        sorted_favs = sorted(
            self.favorites.values(),
            key=lambda x: x.added_at,
            reverse=True
        )
        return sorted_favs[:n]

    def clear_all(self):
        """Clear all favorites"""
        count = len(self.favorites)
        self.favorites = {}
        self._save_favorites()
        print(f"✅ Cleared {count} favorites")

    def get_stats(self) -> Dict:
        """Get favorites statistics"""
        if not self.favorites:
            return {'total_favorites': 0}

        categories = defaultdict(int)
        for fav in self.favorites.values():
            categories[fav.product_category] += 1

        return {
            'total_favorites': len(self.favorites),
            'by_category': dict(categories),
            'oldest': min(self.favorites.values(), key=lambda x: x.added_at).added_at,
            'newest': max(self.favorites.values(), key=lambda x: x.added_at).added_at
        }

print('✅ FavoritesManager class defined')

✅ FavoritesManager class defined


In [31]:
# Test Favorites
favorites = FavoritesManager('U001')

# Add test favorites
print('Adding test favorites...')
test_products = [
    ('P001', 'Blue Summer Dress', 'Dress'),
    ('P045', 'Nike Running Shoes', 'Shoes'),
    ('P123', 'Leather Jacket', 'Jacket'),
    ('P234', 'White T-Shirt', 'Shirt'),
    ('P567', 'Black Jeans', 'Pants'),
]

for prod_id, name, category in test_products:
    favorites.add_favorite(prod_id, name, category)

print('\n❤️  All Favorites:')
for fav in favorites.get_all():
    print(f"  - {fav.product_name} ({fav.product_category})")

print('\n👗 Dress Favorites:')
for fav in favorites.get_by_category('Dress'):
    print(f"  - {fav.product_name}")

print('\n📊 Favorites Statistics:')
print(json.dumps(favorites.get_stats(), indent=2))

print(f"\n❓ Is P001 favorite? {favorites.is_favorite('P001')}")
print(f"❓ Is P999 favorite? {favorites.is_favorite('P999')}")

Adding test favorites...
⚠️  Product P001 already in favorites
⚠️  Product P045 already in favorites
⚠️  Product P123 already in favorites
⚠️  Product P234 already in favorites
⚠️  Product P567 already in favorites

❤️  All Favorites:
  - Blue Summer Dress (Dress)
  - Nike Running Shoes (Shoes)
  - Leather Jacket (Jacket)
  - White T-Shirt (Shirt)
  - Black Jeans (Pants)

👗 Dress Favorites:
  - Blue Summer Dress

📊 Favorites Statistics:
{
  "total_favorites": 5,
  "by_category": {
    "Dress": 1,
    "Shoes": 1,
    "Jacket": 1,
    "Shirt": 1,
    "Pants": 1
  },
  "oldest": "2026-01-04T20:39:52.317912",
  "newest": "2026-01-04T20:39:52.363578"
}

❓ Is P001 favorite? True
❓ Is P999 favorite? False


---

## PART 5: Complete System Test

In [32]:
print('🧪 COMPLETE SYSTEM TEST')
print('='*60)

# Create 3 test users with complete data
test_users = [
    ('U001', 'Alice', 'alice@example.com'),
    ('U002', 'Bob', 'bob@example.com'),
    ('U003', 'Carol', 'carol@example.com'),
]

for uid, name, email in test_users:
    print(f'\n--- Setting up {name} ({uid}) ---')

    # Create or get user
    try:
        user = manager.create_user(uid, name, email)
    except ValueError:
        user = manager.get_user(uid)
        print(f'⚠️  User already exists, using existing profile')

    # Set preferences
    if uid == 'U001':
        prefs = {'style': ['casual', 'sporty'], 'size': 'S', 'colors': ['blue', 'white']}
    elif uid == 'U002':
        prefs = {'style': ['formal'], 'size': 'L', 'colors': ['black', 'gray']}
    else:
        prefs = {'style': ['elegant'], 'size': 'M', 'colors': ['red', 'pink']}

    manager.update_preferences(uid, **prefs)

    # Add search history
    hist = SearchHistory(uid)
    if len(hist.history) < 3:  # Only add if not already there
        hist.add_search(f"test query for {name}", 10, None, 0.15)
        hist.add_search(f"another search {name}", 8, None, 0.12)

    # Add favorites
    favs = FavoritesManager(uid)
    if len(favs.get_all()) < 2:  # Only add if not already there
        favs.add_favorite(f'P00{uid[-1]}', f'Product for {name}', 'Test')

print('\n' + '='*60)
print('📊 SYSTEM SUMMARY')
print('='*60)

print(f"\nTotal users: {len(manager.list_users())}")
print(f"Users: {', '.join(manager.list_users())}")

print('\n📁 Files Created:')
for file in USER_DATA_DIR.glob('*.json'):
    size = file.stat().st_size
    print(f"  - {file.name} ({size} bytes)")

print('\n✅ Complete system test passed!')

🧪 COMPLETE SYSTEM TEST

--- Setting up Alice (U001) ---
⚠️  User already exists, using existing profile
✅ Updated preferences for: U001

--- Setting up Bob (U002) ---
⚠️  User already exists, using existing profile
✅ Updated preferences for: U002
⚠️  Product P002 already in favorites

--- Setting up Carol (U003) ---
⚠️  User already exists, using existing profile
✅ Updated preferences for: U003
⚠️  Product P003 already in favorites

📊 SYSTEM SUMMARY

Total users: 3
Users: U001, U002, U003

📁 Files Created:
  - users.json (1253 bytes)
  - history_U001.json (1662 bytes)
  - favorites_U001.json (896 bytes)
  - history_U002.json (676 bytes)
  - favorites_U002.json (180 bytes)
  - history_U003.json (684 bytes)
  - favorites_U003.json (182 bytes)

✅ Complete system test passed!


---

## PART 6: Save Production Code

In [33]:
# Save classes as Python modules for production use

# This will be done in separate cells to create .py files
print('📝 Saving production code to v2.4-complete/src/')

# We'll create these files in the next cells

📝 Saving production code to v2.4-complete/src/


---

## SUMMARY & NEXT STEPS

In [36]:
print('\n' + '='*60)
print('🎉 User Features Complete!')
print('='*60)

print('\n✅ Completed Features:')
print('  1. ✓ User Profile System')
print('  2. ✓ User Preferences Management')
print('  3. ✓ Search History Tracking')
print('  4. ✓ Favorites System')
print('  5. ✓ JSON Storage')
print('  6. ✓ Complete Testing')

print('\n📊 Statistics:')
print(f'  - Users created: {len(manager.list_users())}')
print(f'  - Search histories: {len(list(USER_DATA_DIR.glob("history_*.json")))}')
print(f'  - Favorites files: {len(list(USER_DATA_DIR.glob("favorites_*.json")))}')

print('\n📁 Output Files:')
print(f'  - v2.4-complete/data/users/users.json')
print(f'  - v2.4-complete/data/users/history_*.json')
print(f'  - v2.4-complete/data/users/favorites_*.json')

print('\n🚀 Next (Day 2):')
print('  - Personalized Recommendations Engine')
print('  - Integration with v2.3 Agent')
print('  - "For You" Page Implementation')



🎉 User Features Complete!

✅ Completed Features:
  1. ✓ User Profile System
  2. ✓ User Preferences Management
  3. ✓ Search History Tracking
  4. ✓ Favorites System
  5. ✓ JSON Storage
  6. ✓ Complete Testing

📊 Statistics:
  - Users created: 3
  - Search histories: 3
  - Favorites files: 3

📁 Output Files:
  - v2.4-complete/data/users/users.json
  - v2.4-complete/data/users/history_*.json
  - v2.4-complete/data/users/favorites_*.json

🚀 Next (Day 2):
  - Personalized Recommendations Engine
  - Integration with v2.3 Agent
  - "For You" Page Implementation
